# Compare cell entry effects for cells expressing alpha2,3 or alpha2,6 sialic acids

In [1]:
import os
import pickle

import altair as alt

import pandas as pd

import yaml

In [2]:
SA23_csv="results/func_effects/averages/293_SA23_entry_func_effects.csv"
SA26_csv="results/func_effects/averages/293_SA26_entry_func_effects.csv"    

In [3]:
# Import data
SA23 = pd.read_csv(SA23_csv)
SA26 = pd.read_csv(SA26_csv)

In [4]:
sialic_acid_entry = pd.merge(SA23, SA26,
                          how='left',
                          on=['site', 'wildtype', 'mutant'],
                         )
sialic_acid_entry

,site,wildtype,mutant,effect_x,effect_std_x,times_seen_x,n_selections_x,effect_y,effect_std_y,times_seen_y,n_selections_y
0,-1,L,*,-6.5170,NaN,5.0,1,-3.7270,NaN,5.0,1.0
1,-1,L,A,-2.7490,2.158000,5.0,2,-2.1630,2.36400,5.0,2.0
2,-1,L,C,0.7591,0.090580,3.5,2,0.6297,0.17480,3.5,2.0
3,-1,L,D,-7.2300,0.309000,5.0,2,-2.1950,2.53000,4.5,2.0
4,-1,L,E,-2.3500,0.001414,3.0,2,-2.2040,0.55080,3.0,2.0
...,...,...,...,...,...,...,...,...,...,...,...
11413,99,P,S,-6.5650,0.805400,2.0,2,-4.2820,1.03700,2.0,2.0
11414,99,P,T,-5.7390,1.419000,4.5,2,-4.0860,0.93060,4.5,2.0
11415,99,P,V,-6.5290,0.420000,4.0,2,-3.9190,0.08061,4.0,2.0
11416,99,P,W,-5.9620,0.330200,2.0,2,-4.2130,0.92910,2.0,2.0


In [10]:
sialic_acid_entry= sialic_acid_entry.loc[sialic_acid_entry['n_selections_y'] >1]
sialic_acid_entry= sialic_acid_entry.loc[sialic_acid_entry['n_selections_x'] >1]

sialic_acid_entry= sialic_acid_entry.rename(columns={"effect_x": "2,3-linked entry","effect_std_x": "2,3-linked entry_std", "effect_y": "2,6-linked entry","effect_std_y": "2,6-linked entry_std"})

In [13]:
alt.data_transformers.disable_max_rows()


slider_1 = alt.binding_range(
    min=sialic_acid_entry["times_seen_x"].min(),
    max=sialic_acid_entry["times_seen_x"].max(),
    step=1,
    name="times seen",
)
selector_1 = alt.selection_point(
    name="SelectorName1",
    fields=["times_seen_x"],
    bind=slider_1,
    value= 3,
)

corr_chart = (
    alt.Chart(sialic_acid_entry)
    .encode(
        x=alt.X(
            "2,3-linked entry",
            title="alpha-2,3 entry",
            axis=alt.Axis(grid=False),
            scale=alt.Scale(domain=[sialic_acid_entry["2,3-linked entry"].min(),
                                    sialic_acid_entry["2,3-linked entry"].max()],
                            padding=10)
        ),
        y=alt.Y(
            "2,6-linked entry",
            title="alpha-2,6 entry",
            axis=alt.Axis(grid=False),
            scale=alt.Scale(domain=[sialic_acid_entry["2,6-linked entry"].min(),
                                    sialic_acid_entry["2,6-linked entry"].max()],
                            padding=10),
        ),
        tooltip=[
            alt.Tooltip(c, format=".3g") if sialic_acid_entry[c].dtypes == float
            else c
            for c in sialic_acid_entry.columns.tolist()
        ],
        opacity=alt.condition(
            alt.FieldOneOfPredicate('site_x', list(range(13, 331))),
            alt.value(1),
            alt.value(0.1)
        ),
        color=alt.condition(
            (alt.datum["2,3-linked entry"] < 0) & (alt.datum["2,6-linked entry"] > 0),
            alt.value('orange'),
            alt.value('black')
        )
    ) 
    .mark_circle(filled=True, size=100, opacity=0.4)
    .resolve_scale(y="independent", x="independent")
    .properties(width=400, height=400)
    .add_params(selector_1)
    .transform_filter(alt.datum["times_seen_x"] > selector_1["times_seen_x"])
)


# Configure axis title font size
corr_chart = corr_chart.configure_axis(
    titleFontSize=16,
    labelFontSize=16
)
corr_chart

alt.Chart(...)